<a href="https://colab.research.google.com/github/george-zakharov/ml_notebooks/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework 9



#### Light: 

Выполнить задачу text summarization для своего набора статей на английском языке.


#### Pro: 

Выполнить задачу text summarization для своего набора статей на русском языке. 
В качестве векторных представлений предложений использовать либо предобученные 
модели https://rusvectores.org/ru/models/ или собственный embending, 
полученный из SVD разложения.

# Load libs

In [36]:
from google.colab import files
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Light (News dataset from Kaggle)

## Load dataset

In [1]:
!pip install kaggle

In [7]:
file = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle

In [9]:
!kaggle datasets download -d asad1m9a9h6mood/news-articles # https://www.kaggle.com/asad1m9a9h6mood/news-articles

  0% 0.00/1.73M [00:00<?, ?B/s]
100% 1.73M/1.73M [00:00<00:00, 117MB/s]


In [10]:
!unzip news-articles.zip

Archive:  news-articles.zip
  inflating: Articles.csv            


In [0]:
df = pd.read_csv('Articles.csv', engine = 'python')

# User defined functions

In [0]:
# function to remove stopwords
def remove_stopwords(sentence):
    sen_new = " ".join([i for i in sentence if i not in stop_words])
    return sen_new

# Dataset exploration

In [15]:
df.head()

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [19]:
df.Article[0]

'KARACHI: The Sindh government has decided to bring down public transport fares by 7 per cent due to massive reduction in petroleum product prices by the federal government, Geo News reported.Sources said reduction in fares will be applicable on public transport, rickshaw, taxi and other means of traveling.Meanwhile, Karachi Transport Ittehad (KTI) has refused to abide by the government decision.KTI President Irshad Bukhari said the commuters are charged the lowest fares in Karachi as compare to other parts of the country, adding that 80pc vehicles run on Compressed Natural Gas (CNG). Bukhari said Karachi transporters will cut fares when decrease in CNG prices will be made.                        \r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n'

# Data transformation

In [0]:
# split the the text in the articles into sentences
sentences = []
for article in df.Article:
  sentences.append(sent_tokenize(article))  

In [27]:
sentences[0]

['KARACHI: The Sindh government has decided to bring down public transport fares by 7 per cent due to massive reduction in petroleum product prices by the federal government, Geo News reported.Sources said reduction in fares will be applicable on public transport, rickshaw, taxi and other means of traveling.Meanwhile, Karachi Transport Ittehad (KTI) has refused to abide by the government decision.KTI President Irshad Bukhari said the commuters are charged the lowest fares in Karachi as compare to other parts of the country, adding that 80pc vehicles run on Compressed Natural Gas (CNG).',
 'Bukhari said Karachi transporters will cut fares when decrease in CNG prices will be made.']

In [0]:
# flatten the list
sentences = [y for x in sentences for y in x]

In [29]:
sentences[0]

'KARACHI: The Sindh government has decided to bring down public transport fares by 7 per cent due to massive reduction in petroleum product prices by the federal government, Geo News reported.Sources said reduction in fares will be applicable on public transport, rickshaw, taxi and other means of traveling.Meanwhile, Karachi Transport Ittehad (KTI) has refused to abide by the government decision.KTI President Irshad Bukhari said the commuters are charged the lowest fares in Karachi as compare to other parts of the country, adding that 80pc vehicles run on Compressed Natural Gas (CNG).'

In [0]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [sentence.lower() for sentence in clean_sentences]

In [34]:
clean_sentences[0]

'karachi  the sindh government has decided to bring down public transport fares by   per cent due to massive reduction in petroleum product prices by the federal government  geo news reported sources said reduction in fares will be applicable on public transport  rickshaw  taxi and other means of traveling meanwhile  karachi transport ittehad  kti  has refused to abide by the government decision kti president irshad bukhari said the commuters are charged the lowest fares in karachi as compare to other parts of the country  adding that   pc vehicles run on compressed natural gas  cng  '

In [0]:
stop_words = stopwords.words('english')

In [0]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [40]:
clean_sentences[0]

'karachi sindh government decided bring public transport fares per cent due massive reduction petroleum product prices federal government geo news reported sources said reduction fares applicable public transport rickshaw taxi means traveling meanwhile karachi transport ittehad kti refused abide government decision kti president irshad bukhari said commuters charged lowest fares karachi compare parts country adding pc vehicles run compressed natural gas cng'

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Extract word vectors
word_embeddings = {}

f = open('/content/drive/My Drive/Colab Notebooks/ML/9/glove.6B.100d.txt', encoding = 'utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    word_embeddings[word] = coefs
f.close()

# Useful links

* https://www.kaggle.com/asad1m9a9h6mood/news-articles